## Dataset Loading and Selection

In [1]:
import os
import pandas as pd
pd.set_option('display.max_columns', None)

fname = os.path.join('CICIDS2017', 'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX_clean.csv')

flows = pd.read_csv(fname)
flows.tail()

c:\program files\python3x64\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (20,21,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,flow_id,source_ip,source_port,destination_ip,destination_port,protocol,timestamp,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,bwd_packet_length_max,bwd_packet_length_min,bwd_packet_length_mean,bwd_packet_length_std,flow_bytes_per_s,flow_packets_per_s,flow_iat_mean,flow_iat_std,flow_iat_max,flow_iat_min,fwd_iat_total,fwd_iat_mean,fwd_iat_std,fwd_iat_max,fwd_iat_min,bwd_iat_total,bwd_iat_mean,bwd_iat_std,bwd_iat_max,bwd_iat_min,fwd_psh_flags,bwd_psh_flags,fwd_urg_flags,bwd_urg_flags,fwd_header_length,bwd_header_length,fwd_packets_per_s,bwd_packets_per_s,min_packet_length,max_packet_length,packet_length_mean,packet_length_std,packet_length_variance,fin_flag_count,syn_flag_count,rst_flag_count,psh_flag_count,ack_flag_count,urg_flag_count,cwe_flag_count,ece_flag_count,down_per_up_ratio,average_packet_size,avg_fwd_segment_size,avg_bwd_segment_size,fwd_header_length.1,fwd_avg_bytes_per_bulk,fwd_avg_packets_per_bulk,fwd_avg_bulk_rate,bwd_avg_bytes_per_bulk,bwd_avg_packets_per_bulk,bwd_avg_bulk_rate,subflow_fwd_packets,subflow_fwd_bytes,subflow_bwd_packets,subflow_bwd_bytes,init_win_bytes_forward,init_win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,label,external_ip
225740,192.168.10.17-192.168.10.50-37758-22-6,192.168.10.50,22,192.168.10.17,37758,6,2017-07-07T17:02:00,85,1,2,0,0,0,0,0.000000,0.00000,0,0,0.0,0.0,0,35294.1,42.5,7.778175,48.0,37.0,0.0,0.0,0.00000,0.0,0.0,48.0,48.0,0.0,48.0,48.0,0,0,0,0,32,64,11764.705880,23529.411760,0,0,0.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,2,0.0,0.000000,0.0,32,0,0,0,0,0,0,1,0,2,0,243,290,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,NaN
225741,192.168.10.14-23.10.108.151-59111-443-6,192.168.10.14,59111,23.10.108.151,443,6,2017-07-07T17:02:00,113,2,0,12,0,6,6,6.000000,0.00000,0,0,0.0,0.0,106195,17699.1,113.0,0.000000,113.0,113.0,113.0,113.0,0.00000,113.0,113.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,17699.115040,0.000000,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0,9.0,6.000000,0.0,40,0,0,0,0,0,0,2,12,0,0,256,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,NaN
225742,192.168.10.17-192.168.10.50-37759-22-6,192.168.10.17,37759,192.168.10.50,22,6,2017-07-07T17:02:00,115,1,1,0,0,0,0,0.000000,0.00000,0,0,0.0,0.0,0,17391.3,115.0,0.000000,115.0,115.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,32,32,8695.652174,8695.652174,0,0,0.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1,0.0,0.000000,0.0,32,0,0,0,0,0,0,1,0,1,0,290,243,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,NaN
225743,162.213.33.50-192.168.10.51-443-59581-6,192.168.10.51,59581,162.213.33.50,443,6,2017-07-07T17:02:00,191310,3,2,148,0,148,0,49.333333,85.44784,0,0,0.0,0.0,773.614,26.1356,47827.5,54961.659690,95485.0,47.0,95825.0,47912.5,67175.85132,95413.0,412.0,95944.0,95944.0,0.0,95944.0,95944.0,0,0,0,0,104,72,15.681355,10.454237,0,148,24.666667,60.420747,3650.666667,0,0,0,1,0,0,0,0,0,29.6,49.333333,0.0,104,0,0,0,0,0,0,3,148,2,0,29200,235,1,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,NaN
225744,192.168.10.17-192.168.10.50-37759-22-6,192.168.10.50,22,192.168.10.17,37759,6,2017-07-07T17:02:00,81,1,2,0,0,0,0,0.000000,0.00000,0,0,0.0,0.0,0,37037,40.5,12.020815,49.0,32.0,0.0,0.0,0.00000,0.0,0.0,49.0,49.0,0.0,49.0,49.0,0,0,0,0,32,64,12345.679010,24691.358020,0,0,0.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,2,0.0,0.000000,0.0,32,0,0,0,0,0,0,1,0,2,0,243,290,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,NaN


There are a lot of additional, calculated field in the CIC dataset. Whilst these are interesting to have for research purposes, I am mostly interested to stay as close to conventional netflows.  
For starters, we'll drop the flow_id as well as the timestamp, as both fields introduce problems and are irrelevant.  
As a continuation, we'll drop most of the remaining calculated field to keep the [Curse of Dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality) at bay and keep complexity and training times under control.

In [2]:
# drop unused fields.
unused_fields = ['flow_id','timestamp','fwd_packet_length_max','fwd_packet_length_min','fwd_packet_length_mean','fwd_packet_length_std','bwd_packet_length_max','bwd_packet_length_min','bwd_packet_length_mean','bwd_packet_length_std','flow_iat_mean','flow_iat_std','flow_iat_max','flow_iat_min','fwd_iat_total','fwd_iat_mean','fwd_iat_std','fwd_iat_max','fwd_iat_min','bwd_iat_total','bwd_iat_mean','bwd_iat_std','bwd_iat_max','bwd_iat_min','min_packet_length','max_packet_length','packet_length_mean','packet_length_std','packet_length_variance','average_packet_size','avg_fwd_segment_size','avg_bwd_segment_size','fwd_avg_bytes_per_bulk','fwd_avg_packets_per_bulk','fwd_avg_bulk_rate','bwd_avg_bytes_per_bulk','bwd_avg_packets_per_bulk','bwd_avg_bulk_rate','active_mean','active_std','active_max','active_min','idle_mean','idle_std','idle_max','idle_min','min_seg_size_forward']
flows.drop(unused_fields, axis=1, inplace=True)
flows.head()

,source_ip,source_port,destination_ip,destination_port,protocol,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,flow_bytes_per_s,flow_packets_per_s,fwd_psh_flags,bwd_psh_flags,fwd_urg_flags,bwd_urg_flags,fwd_header_length,bwd_header_length,fwd_packets_per_s,bwd_packets_per_s,fin_flag_count,syn_flag_count,rst_flag_count,psh_flag_count,ack_flag_count,urg_flag_count,cwe_flag_count,ece_flag_count,down_per_up_ratio,fwd_header_length.1,subflow_fwd_packets,subflow_fwd_bytes,subflow_bwd_packets,subflow_bwd_bytes,init_win_bytes_forward,init_win_bytes_backward,act_data_pkt_fwd,label,external_ip
0,192.168.10.16,41936,199.244.48.55,443,6,143347,47,60,1325,108751,767898.8748,746.4404557,0,0,0,0,1200,1928,327.875714,418.564742,0,0,0,1,0,0,0,0,1,1200,47,1325,60,108751,29200,61,30,BENIGN,NaN
1,192.168.10.16,42970,54.210.195.63,80,6,50905,1,1,0,0,0,39.28887143,0,0,0,0,32,32,19.644436,19.644436,0,0,0,0,1,1,0,0,1,32,1,0,1,0,251,110,0,BENIGN,NaN
2,192.168.10.16,41944,199.244.48.55,443,6,143899,46,58,1325,110185,774918.5192,722.7291364,0,0,0,0,1168,1864,319.668656,403.060480,0,0,0,1,0,0,0,0,1,1168,46,1325,58,110185,29200,61,30,BENIGN,NaN
3,192.168.10.17,12886,192.168.10.3,53,17,313,2,2,90,206,945686.901,12779.55272,0,0,0,0,40,64,6389.776358,6389.776358,0,0,0,0,0,0,0,0,1,40,2,90,2,206,-1,-1,1,BENIGN,NaN
4,192.168.10.16,41942,199.244.48.55,443,6,142605,45,58,1325,108751,771894.3936,722.274815,0,0,0,0,1136,1864,315.556958,406.717857,0,0,0,1,0,0,0,0,1,1136,45,1325,58,108751,29200,61,30,BENIGN,NaN


## Data Encoding

There's still a problem: How can we encode IP addresses in a way that the neural network can make use of them while preserving the hierarchical information they contain?  
Encoding IPs through One Hot let's comlexity and training times explode, so for now I am splitting each IP into its four octet pairs and interpret them as numbers.  
Maybe there's a better way to represent them (especially because I am only able to encode IPv4 right now)

In [3]:
# https://stackoverflow.com/questions/14745022/how-to-split-a-column-into-two-columns
# FIXME: Right now, only IPv4 (4 octets)

# Split the String representation of the IP into it's four octects, which are delimited by a dot
flows['source_ip_o1'],flows['source_ip_o2'],flows['source_ip_o3'],flows['source_ip_o4'] = flows['source_ip'].str.split('.').str
flows['destination_ip_o1'],flows['destination_ip_o2'],flows['destination_ip_o3'],flows['destination_ip_o4'] = flows['destination_ip'].str.split('.').str
flows['external_ip_o1'],flows['external_ip_o2'],flows['external_ip_o3'],flows['external_ip_o4'] = flows['external_ip'].str.split('.').str

# After completion, drop the initial columns, as they aren't needed anymore
flows.drop(['source_ip'], axis=1, inplace=True)
flows.drop(['destination_ip'], axis=1, inplace=True)
flows.drop(['external_ip'], axis=1, inplace=True)

# Finally, let's inspect the outcome
flows.head()

,source_port,destination_port,protocol,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,flow_bytes_per_s,flow_packets_per_s,fwd_psh_flags,bwd_psh_flags,fwd_urg_flags,bwd_urg_flags,fwd_header_length,bwd_header_length,fwd_packets_per_s,bwd_packets_per_s,fin_flag_count,syn_flag_count,rst_flag_count,psh_flag_count,ack_flag_count,urg_flag_count,cwe_flag_count,ece_flag_count,down_per_up_ratio,fwd_header_length.1,subflow_fwd_packets,subflow_fwd_bytes,subflow_bwd_packets,subflow_bwd_bytes,init_win_bytes_forward,init_win_bytes_backward,act_data_pkt_fwd,label,source_ip_o1,source_ip_o2,source_ip_o3,source_ip_o4,destination_ip_o1,destination_ip_o2,destination_ip_o3,destination_ip_o4,external_ip_o1,external_ip_o2,external_ip_o3,external_ip_o4
0,41936,443,6,143347,47,60,1325,108751,767898.8748,746.4404557,0,0,0,0,1200,1928,327.875714,418.564742,0,0,0,1,0,0,0,0,1,1200,47,1325,60,108751,29200,61,30,BENIGN,192,168,10,16,199,244,48,55,NaN,NaN,NaN,NaN
1,42970,80,6,50905,1,1,0,0,0,39.28887143,0,0,0,0,32,32,19.644436,19.644436,0,0,0,0,1,1,0,0,1,32,1,0,1,0,251,110,0,BENIGN,192,168,10,16,54,210,195,63,NaN,NaN,NaN,NaN
2,41944,443,6,143899,46,58,1325,110185,774918.5192,722.7291364,0,0,0,0,1168,1864,319.668656,403.060480,0,0,0,1,0,0,0,0,1,1168,46,1325,58,110185,29200,61,30,BENIGN,192,168,10,16,199,244,48,55,NaN,NaN,NaN,NaN
3,12886,53,17,313,2,2,90,206,945686.901,12779.55272,0,0,0,0,40,64,6389.776358,6389.776358,0,0,0,0,0,0,0,0,1,40,2,90,2,206,-1,-1,1,BENIGN,192,168,10,17,192,168,10,3,NaN,NaN,NaN,NaN
4,41942,443,6,142605,45,58,1325,108751,771894.3936,722.274815,0,0,0,0,1136,1864,315.556958,406.717857,0,0,0,1,0,0,0,0,1,1136,45,1325,58,108751,29200,61,30,BENIGN,192,168,10,16,199,244,48,55,NaN,NaN,NaN,NaN


The labels of the dataset (as in: *Benign*, *DDoS*, *Portscan*, etc) are converted into a list of integers.  
A humble example (not representative):  

|Label         | Value          |
|------------- |---------:|
|Benign      | 1|
|DDoS        | 2|
|Portscan    | 3|

In [4]:
import numpy as np
from keras.preprocessing.text import Tokenizer
# tokenize the LABELS
label_tokenizer = Tokenizer(num_words=20) 
label_tokenizer.fit_on_texts(flows['label'])

# Run the fitted tokenizer on the label column and save the encoded data as dataframe
enc_labels = label_tokenizer.texts_to_sequences(flows['label'])
enc_labels = np.concatenate(enc_labels).ravel()

# finally, drop the label column
flows.drop(['label'], axis=1, inplace=True)

Using TensorFlow backend.


While we're at it, we make sure to never have any float values exceeding +/- infinity as well as NaN values.  
These are all replaces by zeros, which is a temporary fix and definitely a FIXME for the future

In [5]:
# weed out all NaN and infinite values
flows.replace([np.inf, -np.inf], np.nan)
flows.fillna(inplace=True, value=0) # FIXME: 0 for now, find a better way

As Keras seems to be a bit picky about the presented datatypes, we'll convert the Pandas DataFrame into it's underlying representation of Numpy-Arrays and work with these from this point onwars.

In [6]:
flows_nd = flows.astype('float64').values

In [7]:
# as the pandas infinity stuff is seemingly not enough, check the numpy array once again
from numpy import inf
flows_nd[flows_nd == -inf] = 0
flows_nd[flows_nd == inf] = 0

In [8]:
print("Data has NaN:",np.any(np.isnan(flows_nd)))
print("Data has only finite values",np.all(np.isfinite(flows_nd)))

Data has NaN: False
Data has only finite values True


## Data Normalization and Finishing Touches

Best performance is achieved if all values are normalized. In this approach I am using [sklearn's MinMaxScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html), which implements feature scaling through MinMax-Normalization (Rescaling).

In [9]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
flows_scaled = min_max_scaler.fit_transform(flows_nd)

Let's have a final glance at a single entry of our dataset:

In [10]:
print(flows_scaled[0])

[6.39912107e-01 6.76005616e-03 3.52941176e-01 1.19456728e-03
 2.38218540e-02 2.03942896e-02 7.23996241e-03 2.10254689e-02
 6.13251627e-03 4.00149288e-01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 3.04599452e-02 3.27601441e-02
 1.09291905e-04 2.09282371e-04 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.42857143e-01 3.04599452e-02
 2.38218540e-02 7.23996241e-03 2.03942896e-02 2.10254689e-02
 4.45571899e-01 9.46044922e-04 1.55359917e-02 8.60360360e-01
 6.58823529e-01 3.92156863e-02 6.27450980e-02 7.79527559e-01
 9.56862745e-01 1.88235294e-01 2.15686275e-01 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00]


In [11]:
print("Shape of the final netflow dataset:", flows_scaled.shape)
print("Outer type:", type(flows_scaled))
print("Single entry type:", type(flows_scaled[0]))

Shape of the final netflow dataset: (225745, 47)
Outer type: <class 'numpy.ndarray'>
Single entry type: <class 'numpy.ndarray'>


Last but not least, the [Keras Embedding Layer](https://keras.io/layers/embeddings/#embedding) expects a maximum vocabulary size, which we can simply calculate by finding max() in our scaled data:

In [12]:
#find the maximum vocabulary size
voc_size = (flows_scaled.max()+1).astype('int64')
print("Maximum vocabulary size:", voc_size)

Maximum vocabulary size: 2


## Building and Training the Model

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(voc_size, 32)) 
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(flows_scaled, enc_labels,
                    epochs=1,
                    batch_size=64,
                    validation_split=0.2)

Train on 180596 samples, validate on 45149 samples
Epoch 1/1
180596/180596 [==============================] - 142s 789us/step - loss: -3.1203 - acc: 0.7971 - val_loss: -1.8527 - val_acc: 0.8838
